In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import random
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import shutil

sys.path.append('../')
from meta_fusion.benchmarks import *
from meta_fusion.methods import *
from meta_fusion.models import *
from meta_fusion.utils import *
from meta_fusion.third_party import *
from meta_fusion.synthetic_data import PrepareSyntheticData
from meta_fusion.config import *
from meta_fusion.methodsextra import *


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/anaconda3/envs/fusion_stable310/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/anaconda3/envs/fusion_stable310/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/anaconda3/envs/fusion_stable310/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/envs/fusion_stable310/lib/python3.10/site-packages/traitlets/config/application.py",

In [2]:
#########################
# Experiment parameters #
#########################
if True:
    # Parse input arguments
    print ('Number of arguments:', len(sys.argv), 'arguments.')
    print ('Argument List:', str(sys.argv))
    if len(sys.argv) != 2:
        print("Error: incorrect number of parameters.")
        quit()

    seed = 1234
    print(seed)

# Fixed data parameters
repetition=1

# Data model parameters
n = 2000
dim_modalities = [500, 400]
dim_latent = [20, 30, 0]
noise_ratios = [0.4, 0.4]
trans_type = ["linear", "linear", "linear"]
mod_prop = [1, 1, 0, 0]
interactive_prop = 0

# mod_outs = [[0, 200, 300, 400, 500], [0, 100, 200, 300, 400]]
mod_outs = [[0, 500], [0, 400]]
num_modalities = 2
combined_hiddens = [128, 64]
mod_hiddens = [[256], [256]]

# data parameters
data_name = 'regression'
exp_name = data_name + "_" + "linear_early"
output_dim = 1  # specify the output dimension for regression


extractor_type = 'separate'
separate=True
is_mod_static=[False]*num_modalities
freeze_mod_extractors=[False]*num_modalities

# Load default model configurations 
config = load_config('../experiments_synthetic/config.json')
extractor_config = load_config('../experiments_synthetic/config_extractor.json')

# Model files directory
ckpt_dir = f"./checkpoints/{exp_name}/seed{seed}/"
config['ckpt_dir'] = extractor_config['ckpt_dir'] = ckpt_dir

# Update other training parameters
config['output_dim'] = extractor_config['output_dim'] = output_dim
config["init_lr"] = 0.001
config["ensemble_methods"] = [
        "simple_average",
        "weighted_average",
        "meta_learner",
        "greedy_ensemble"
        ]
extractor_config["init_lr"] = [0.001] * num_modalities
extractor_config["weight_decay"] = [0] * num_modalities

#####################
#    Load Dataset   #
#####################
data_preparer = PrepareSyntheticData(data_name = data_name, test_size = 0.2, val_size = 0.2)
print(f"Finished generating {exp_name} dataset.")
sys.stdout.flush() 


###############
# Output file #
###############i:
outdir = f"./results/{exp_name}/"
os.makedirs(outdir, exist_ok=True)
outfile_name = f"seed{seed}"
outfile = outdir + outfile_name + ".txt"
print("Output file: {:s}".format(outfile), end="\n")
sys.stdout.flush()


# Header for results file
def add_header(results):
    results['extractor']=extractor_type
    results['weight_type'] = config['divergence_weight_type'] 
    return results




Number of arguments: 2 arguments.
Argument List: ['/opt/anaconda3/envs/fusion_stable310/lib/python3.10/site-packages/ipykernel_launcher.py', '--f=/Users/parnian/Library/Jupyter/runtime/kernel-v3c8604cc7a5aa2caed39cf4980d1fddda8e882702.json']
1234
Finished generating regression_linear_early dataset.
Output file: ./results/regression_linear_early/seed1234.txt


In [5]:
#####################
# Define Experiment #
#####################
def run_single_experiment(config, extractor_config, n, random_state, 
                          mod_outs, combined_hiddens, mod_hiddens,
                          separate, is_mod_static, freeze_mod_extractors,
                          run_oracle=False, run_coop=True, run_all_at_once=False):


    config['random_state'] = random_state
    extractor_config['random_state'] = random_state
    res_list = []
    best_rho = {}
    cohort_pairs = {}
    ens_idxs = {}
    cluster_idxs = {}


    #----------------#
    # Split dataset  #
    #----------------#
    train_loader, val_loader, test_loader, oracle_train_loader, oracle_val_loader, oracle_test_loader =\
    data_preparer.get_data_loaders(n, trans_type=trans_type, mod_prop=mod_prop, 
                                    interactive_prop = interactive_prop,
                                    dim_modalities=dim_modalities, dim_latent=dim_latent,
                                    noise_ratios=noise_ratios, random_state=random_state)
    # Get data info
    data_info = data_preparer.get_data_info()
    n = data_info[1]
    n_train = data_info[2]
    n_val = data_info[3]
    n_test = data_info[4]

    print(f"Finished splitting {data_name} dataset. Data information are summarized below:\n"
            f"Modality dimensions: {dim_modalities}\n"
            f"Data size: {n}\n"
            f"Train size: {n_train}\n"
            f"Val size: {n_val}\n"
            f"Test size: {n_test}")
    sys.stdout.flush() 

    #------------------#
    # Benchmark models #
    #------------------#
    bm_extractor = Extractors([[d,0] for d in dim_modalities], dim_modalities, train_loader, val_loader)
    _ = bm_extractor.get_dummy_extractors()
    bm_cohort = Cohorts(extractors=bm_extractor, combined_hidden_layers=combined_hiddens, output_dim=output_dim)

    if run_oracle:
        oracle_dims = [dim_latent[0], dim_latent[1]+dim_latent[2]]
        oracle_extractor = Extractors([[d,0] for d in oracle_dims], oracle_dims, oracle_train_loader, oracle_val_loader)
        _ = oracle_extractor.get_dummy_extractors()
        oracle_cohort = Cohorts(extractors=oracle_extractor, combined_hidden_layers=combined_hiddens, output_dim=output_dim)

    #----------------------------#
    # Proposed model: Meta Fuse  #
    #----------------------------#
    meta_extractor = Extractors(mod_outs, dim_modalities, train_loader, val_loader)
    if (extractor_type == 'encoder') or (extractor_type == 'separate'):
        _ = meta_extractor.get_encoder_extractors(mod_hiddens, separate=separate, config=extractor_config)
    elif extractor_type == 'PCA':
        _ = meta_extractor.get_PCA_extractors()
    meta_cohort = Cohorts(extractors=meta_extractor, combined_hidden_layers=combined_hiddens, output_dim=output_dim,
                            is_mod_static=is_mod_static, freeze_mod_extractors=freeze_mod_extractors)


    #------------------------------#
    #  Train and test benchmarks   #
    #------------------------------#
    bm_models = bm_cohort.get_cohort_models()
    _, bm_dims = bm_cohort.get_cohort_info()
    bm = Benchmarks(config, bm_models, bm_dims, [train_loader, val_loader])
    bm.train()
    res = bm.test(test_loader)
    res_list.append(res)
    print(f"Finished running basic benchmarks!")

    if run_oracle:
        oracle_config = config
        oracle_config["init_lr"] = 0.001
        oracle_models = oracle_cohort.get_cohort_models()
        _, oracle_dims = oracle_cohort.get_cohort_info()
        oracle = Benchmarks(config, oracle_models, oracle_dims, [oracle_train_loader, oracle_val_loader])
        oracle.train()
        res = oracle.test(oracle_test_loader)
        res = {f"oracle_{key}": value for key, value in res.items()}
        res_list.append(res)
        print(f"Finished running oracle benchmarks!")
        
    if run_coop:
        bm_models = bm_cohort.get_cohort_models()
        _, bm_dims = bm_cohort.get_cohort_info()    
        coop = Coop(config, bm_models, bm_dims, [train_loader, val_loader])
        coop.train()
        res = coop.test(test_loader)
        res_list.append(res)
        best_rho['coop'] = coop.best_rho
        print(f"Finished running coop!")


    #------------------------------#
    #  Train and test Meta Fuse   #
    #------------------------------#
    cohort_models = meta_cohort.get_cohort_models()
    _, dim_pairs = meta_cohort.get_cohort_info()
    ###### Only change the two following.
    metafuse = Trainer(config, cohort_models, [train_loader, val_loader]) # New trainer function. 
    metafuse.train() 
    res = metafuse.test(test_loader) # No need to change test: simple_averaging in test_regresion() also # performance of each student on the test data, cohort_accuracy: automaticall printed and stored. 
    res = {f"metafusion_{k}": v for k, v in res.items()}
    res_list.append(res)
    metafuse.train_ablation() # This is just late fusion with student cohort.
    res = metafuse.test_ablation(test_loader) # I don't need this, no need to have different rhos.
    res = {f"indep_{k}": v for k, v in res.items()}
    res_list.append(res)

    best_rho['metafusion'] = metafuse.best_rho
    cohort_pairs['metafusion'] = dim_pairs
    cohort_pairs['indep'] = dim_pairs

    if "greedy_ensemble" in config["ensemble_methods"]:
        ens_idxs['metafusion_greedy_ensemble'] = metafuse.ens_idxs  

    if config['divergence_weight_type'] == "clustering":
        cluster_idxs['metafusion'] = metafuse.cluster_idxs

    print(f"Finished running meta fusion!")


    #----------------------------#
    # Proposed model: Joint train#
    #----------------------------#
    joint_extractor = Extractors(mod_outs, dim_modalities, train_loader, val_loader)
    if (extractor_type == 'encoder') or (extractor_type == 'separate'):
        _ = joint_extractor.get_encoder_extractors(mod_hiddens, separate=separate, config=extractor_config)
    elif extractor_type == 'PCA':
        _ = joint_extractor.get_PCA_extractors()
    joint_cohort = Cohorts(extractors=joint_extractor, combined_hidden_layers=combined_hiddens, output_dim=output_dim,
                          is_mod_static=is_mod_static, freeze_mod_extractors=freeze_mod_extractors)

    # ------------------------------#
    #  Train and test Joint train  #
    # ------------------------------#
    cohort_models = joint_cohort.get_cohort_models()
    _, dim_pairs = joint_cohort.get_cohort_info()
    ###### Only change the two following.
    jointmodel = Trainer_Joint(config, cohort_models, [train_loader, val_loader]) # New trainer function. 
    jointmodel.train('marginal') 
    res = jointmodel.test(test_loader) # No need to change test: simple_averaging in test_regresion() also # performance of each student on the test data, cohort_accuracy: automaticall printed and stored. 
    res = {f"jointlearning_{k}": v for k, v in res.items()}
    res_list.append(res)
    cohort_pairs['cohort'] = dim_pairs

    if "greedy_ensemble" in config["ensemble_methods"]:
        ens_idxs['jointlearning_greedy_ensemble'] = jointmodel.ens_idxs  


    print(f"Finished running joint fusion!")

    #----------------------------#
    # Proposed model: Negative Correlation Learning#
    #----------------------------#
    joint_extractor = Extractors(mod_outs, dim_modalities, train_loader, val_loader)
    if (extractor_type == 'encoder') or (extractor_type == 'separate'):
        _ = joint_extractor.get_encoder_extractors(mod_hiddens, separate=separate, config=extractor_config)
    elif extractor_type == 'PCA':
        _ = joint_extractor.get_PCA_extractors()
    NCL_cohort = Cohorts(extractors=joint_extractor, combined_hidden_layers=combined_hiddens, output_dim=output_dim,
                            is_mod_static=is_mod_static, freeze_mod_extractors=freeze_mod_extractors)

    #------------------------------#
    #  Train and test NCL train  #
    #------------------------------#
    NCL_models = NCL_cohort.get_cohort_models()
    _, dim_pairs = NCL_cohort.get_cohort_info()
    ###### Only change the two following.
    ncl_model = Trainer_NCL(config, NCL_models, [train_loader, val_loader]) # New trainer function. 
    ncl_model.train() 
    res = ncl_model.test(test_loader) # No need to change test: simple_averaging in test_regresion() also # performance of each student on the test data, cohort_accuracy: automaticall printed and stored. 
    res = {f"ncl_{k}": v for k, v in res.items()}
    res_list.append(res)
    cohort_pairs['ncl'] = dim_pairs
    best_rho['ncl'] = ncl_model.best_rho

    if "greedy_ensemble" in config["ensemble_methods"]:
        ens_idxs['ncl_greedy_ensemble'] = ncl_model.ens_idxs  


    print(f"Finished running NCL fusion!")

    #----------------------------#
    # Proposed model: Shapley train#
    #----------------------------#
    joint_extractor = Extractors(mod_outs, dim_modalities, train_loader, val_loader)
    if (extractor_type == 'encoder') or (extractor_type == 'separate'):
        _ = joint_extractor.get_encoder_extractors(mod_hiddens, separate=separate, config=extractor_config)
    elif extractor_type == 'PCA':
        _ = joint_extractor.get_PCA_extractors()
    joint_cohort = Cohorts(extractors=joint_extractor, combined_hidden_layers=combined_hiddens, output_dim=output_dim,
                        is_mod_static=is_mod_static, freeze_mod_extractors=freeze_mod_extractors)

    #------------------------------#
    #  Train and test shapley train  #
    #------------------------------#
    cohort_models = joint_cohort.get_cohort_models()
    _, dim_pairs = joint_cohort.get_cohort_info()
    ###### Only change the two following.
    jointmodel = Trainer_Joint(config, cohort_models, [train_loader, val_loader]) # New trainer function. 
    jointmodel.train('shapley') 
    res = jointmodel.test(test_loader) # No need to change test: simple_averaging in test_regresion() also # performance of each student on the test data, cohort_accuracy: automaticall printed and stored. 
    res = {f"shapley_{k}": v for k, v in res.items()}
    res_list.append(res)
    cohort_pairs['cohort'] = dim_pairs

    if "greedy_ensemble" in config["ensemble_methods"]:
        ens_idxs['shapley_greedy_ensemble'] = jointmodel.ens_idxs  


    print(f"Finished running shapley fusion!")        

    results = []
    for i, res in enumerate(res_list):
        for method, val in res.items():
            results.append({'Method': method, 'Test_metric': val, 
                            'best_rho':best_rho.get(method.split('_')[0]), 'cohort_pairs':cohort_pairs.get(method.split('_')[0]),
                            'ensemble_idxs': ens_idxs.get(method), 'cluster_idxs': cluster_idxs.get(method.split('_')[0])})

    results = pd.DataFrame(results)
    results['random_state']=random_state
    results["dim_modalities"] = [dim_modalities] * len(results)
    results['n'] = n
    results['n_train'] = n_train
    results['n_val'] = n_val
    results['n_test'] = n_test 

    return results




In [6]:
#####################
#  Run Experiments  #
#####################
results = []

for i in tqdm(range(1, repetition+1), desc="Repetitions", leave=True, position=0):
    print(f'Running with repetition {i}...')
    random_state = repetition * (seed-1) + i
    # print(random_state)
    set_random_seed(random_state)

    # Run experiment
    tmp = run_single_experiment(config, extractor_config, n, random_state, 
                                mod_outs, combined_hiddens, mod_hiddens,
                                separate, is_mod_static, freeze_mod_extractors,
                                run_oracle=False, run_coop=True, run_all_at_once=False)
    
    results.append(tmp)


results = pd.concat(results, ignore_index=True)

add_header(results)

Repetitions:   0%|          | 0/1 [00:00<?, ?it/s]

Running with repetition 1...
Finished splitting regression dataset. Data information are summarized below:
Modality dimensions: [500, 400]
Data size: 2000
Train size: 1280
Val size: 320
Test size: 400
Start training benchmark models...
Training with disagreement penalty = 0

Epoch: 1/10 - LR: 0.001000


100%|██████████| 1280/1280 [00:00<00:00, 3226.35it/s]


model_1: train loss: 130.209, train task loss: 130.209 - val loss: 104.447, val task loss: 104.447 [*] Best so far
model_2: train loss: 130.523, train task loss: 130.523 - val loss: 107.967, val task loss: 107.967 [*] Best so far
model_3: train loss: 121.284, train task loss: 121.284 - val loss: 79.854, val task loss: 79.854 [*] Best so far

Epoch: 2/10 - LR: 0.001000


100%|██████████| 1280/1280 [00:00<00:00, 2915.82it/s]


...

1280it [00:00, 3387.28it/s]


meta_learner: train task loss: 13.737 - val task loss: 12.203 [*] Best so far


1280it [00:00, 3317.42it/s]
Repetitions: 100%|██████████| 1/1 [01:09<00:00, 69.33s/it]

meta_learner: train task loss: 11.145 - val task loss: 11.343 [*] Best so far
Done!
Selecting greedy ensemble on the best cohort...
Pruned 1 worst models, keeping 2 models
Initial best models: [0, 2] with losses: [tensor(9.0733, grad_fn=<MseLossBackward0>), tensor(63.0920, grad_fn=<MseLossBackward0>)]
Done!
Method: (simple_average), Test_MSE: 9.408285140991211
Method: (weighted_average), Test_MSE: 5.01217794418335
Method: (meta_learner), Test_MSE: 10.518463134765625
Method: (greedy_ensemble), Test_MSE: 6.667391300201416
Method: (best_single), Test_MSE: 8.215972900390625
Method: (cohort), Test_MSE: [8.215972900390625, 84.87994384765625, 64.48446655273438]
Finished running shapley fusion!


,Method,Test_metric,best_rho,cohort_pairs,ensemble_idxs,cluster_idxs,random_state,dim_modalities,n,n_train,n_val,n_test,extractor,weight_type
0,modality_1,73.765739,NaN,None,None,None,1234,"[500, 400]",2000,1280,320,400,separate,clustering
1,modality_2,58.633133,NaN,None,None,None,1234,"[500, 400]",2000,1280,320,400,separate,clustering
2,early_fusion,3.935762,NaN,None,None,None,1234,"[500, 400]",2000,1280,320,400,separate,clustering
3,late_fusion,37.302422,NaN,None,None,None,1234,"[500, 400]",2000,1280,320,400,separate,clustering
4,coop,4.119238,0.0,None,None,None,1234,"[500, 400]",2000,1280,320,400,separate,clustering
5,metafusion_simple_average,17.986481,0.0,"[(500, 400), (500, 0), (0, 400)]",None,[0],1234,"[500, 400]",2000,1280,320,400,separate,clustering
6,metafusion_weighted_average,4.748543,0.0,"[(500, 400), (500, 0), (0, 400)]",None,[0],1234,"[500, 400]",2000,1280,320,400,separate,clustering
7,metafusion_meta_learner,4.266782,0.0,"[(500, 400), (500, 0), (0, 400)]",None,[0],1234,"[500, 400]",2000,1280,320,400,separate,clustering
8,metafusion_greedy_ensemble,4.509246,0.0,"[(500, 400), (500, 0), (0, 400)]","[0, 2]",[0],1234,"[500, 400]",2000,1280,320,400,separate,clustering
9,metafusion_best_single,4.197611,0.0,"[(500, 400), (500, 0), (0, 400)]",None,[0],1234,"[500, 400]",2000,1280,320,400,separate,clustering


In [6]:
#####################
#    Save Results   #
#####################
results.to_csv(outfile, index=False)
print("\nResults written to {:s}\n".format(outfile))
sys.stdout.flush()

# After the job is done, remove the model directory to free up space
if os.path.exists(ckpt_dir):
    print(f"Deleting the model checkpoint directory: {ckpt_dir}")
    shutil.rmtree(ckpt_dir)
    print(f"Model checkpoint directory {ckpt_dir} has been deleted.")



Results written to ./results/regression_linear_early/seed1234.txt

